In [9]:
import pandas as pd
import scipy as sp
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate , train_test_split , StratifiedShuffleSplit
from sklearn import   linear_model, metrics
from sklearn.linear_model import SGDClassifier as SGD

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from statistics import mean
from scipy import sparse

import hdbscan

In [2]:
segment_directory = "data2/segmentation.csv"

segment = pd.read_csv(segment_directory, header=0 )

In [5]:
data = segment[['Monetary' ,'Frequency','factor_1','factor_2' ,'factor_3' , 'factor_4']]

In [7]:
from MulticoreTSNE import MulticoreTSNE as TSNE_multi
import time

# tfidf_df = DataFrame(tfidf.todense())
# tfidf_df.index.name = 'docs'
# cols = ['parm' + str(i) for i in range(0, tfidf_df.shape[1])]
# tfidf_df.columns = pd.Series(cols)

# tfidf_df.drop_duplicates(inplace=True)

# tfidf_no_dup = sparse.csr_matrix(tfidf_df.to_numpy())

# tfidf_df_subset = tfidf_df.loc[:, 'parm0':].copy()

# tfidf_subset = tfidf_df_subset.to_numpy()
data_numpy = data.to_numpy()

time_start = time.time()
tsne = TSNE_multi(n_components=2, verbose=1, perplexity=50, n_iter=2000, random_state=42, n_jobs=-1)

tsne_results = tsne.fit_transform(data_numpy)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))


# tfidf_df['tsne_coord_1'] = tsne_results[:,0]
# tfidf_df['tsne_coord_2'] = tsne_results[:,1]

t-SNE done! Time elapsed: 90.62316799163818 seconds


NameError: name 'sparse' is not defined

In [10]:
tsne_sparse = sparse.csr_matrix(tsne_results)

In [15]:
tsne_df = pd.DataFrame()
tsne_df['tsne_1'] = tsne_results[:,0]
tsne_df['tsne_2'] = tsne_results[:,1]

In [31]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=2000, min_samples=30).fit(tsne_df)
clusterer.labels_.max()

2

In [32]:
clusters = clusterer.labels_

n_clusters_ = len(set(clusters)) - (1 if -1 in clusters else 0)
n_noise_ = list(clusters).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

Estimated number of clusters: 3
Estimated number of noise points: 6804


In [33]:
if 'cluster' in tsne_df.columns:
    tsne_df = tsne_df.drop(columns='cluster')
tsne_df.insert(0, 'cluster', clusters)

In [36]:
from cycler import cycler
import random
 
def generate_rgb_list(n):
    ret = []
    r = int(random.random() * 255)
    g = int(random.random() * 255)
    b = int(random.random() * 255)
    step = 255 / n
    for i in range(n):
        r += step
        g += step
        b += step
        r = int(r) % 255
        g = int(g) % 255
        b = int(b) % 255
        ret.append((r/255.0,g/255.0,b/255.0)) 
    return ret

def rgb_to_hex(rgb):
    return matplotlib.colors.to_hex(rgb)

colors = generate_rgb_list(clusters.max()+2)

legend_list = [i for i in range(-1, clusters.max() + 1)]
legend_str_list = ['Кластер ' + str(i) for i in range(-1, clusters.max() + 1)]
legend_str_list[0] = 'Шум'

color_map = {legend_list[i]:colors[i] for i in range(clusters.max() + 2)}
color_map[-1] = (0.85, 0.85, 0.85)
# print(color_map)
cvec = [color_map[cluster] for cluster in clusters]
# print(cvec)

# marker_map = {legend_list[i]:'o' for i in range(clusters.max() + 2)}
# marker_map[-1] = 'D'
# print(marker_map)

# scatter_plt_list = [plt.scatter(tfidf_df['tsne_coord_1'], tfidf_df['tsne_coord_2'], 
#                                 color=color_map[i]) for i in range(-1, clusters.max() + 1)]

scatter_plt_list = [plt.scatter(tsne_df[:,1], tsne_df[:,2], 
                                color=color_map[i]) for i in range(-1, clusters.max() + 1)]


fig = plt.figure(num='Кластеризованный датасет', figsize=(16,10))

plt.scatter(tsne_df[:,1],lsa_visual_results[:,2], c=cvec)
plt.title('Кластеризованный датасет по DBSCAN')
plt.xlabel('Преобразованные координаты x')
plt.ylabel('Преобразованные координаты y')

legend_tuple = (i for i in scatter_plt_list)

plt.legend(legend_tuple, (i for i in legend_str_list))
plt.show()
# sns.scatterplot(
#     x="tsne_coord_1", y="tsne_coord_2",
#     hue="cluster",
#     palette=sns.color_palette("Greens", n_colors=clusters.max()+2), 
#     data=tfidf_df,
#     legend="full",
#     alpha=0.5
# )

# plt.figure(num='Некластеризованный/оригинальный датасет', figsize=(16,10))
# sns.scatterplot(
#     x="tsne-2d-one", y="tsne-2d-two",
#     hue="rating",
#     palette=sns.color_palette("hls", 2), 
#     data=tfidf_uncl_df,
#     legend="full",
#     alpha=0.5
# )

TypeError: '(slice(None, None, None), 1)' is an invalid key